# Mapping 

Transportation is about getting from place A to place B.  Therefore, most transportation data has a spatial component to it.  It is nice to be able to put these data on a map and see what is going on.  It is even better if we can put it on a map and interact with the data.  It would be even cooler if we could put our interactive map on a website to show it off!

To do this, we are going to use a package called folium.  You can find the documentation here: 

https://folium.readthedocs.io/en/latest/

And access it on github here: 

https://github.com/python-visualization/folium


### Credits

This lesson draws from the folium quickstart notebook, and from Vik Paruchuri DataQuest lesson: 

https://www.dataquest.io/blog/python-data-visualization-libraries/

### A side note on static mapping

Sometimes you may want to create a static map instead of an interactive map.  Interactive maps are nice for exploring your data, but static maps work well for an image that you can insert into a paper.  If you want to create static maps, then basemap is a good tool.  Here is a nice lesson focused on mapping earthquake activity: 

http://introtopython.org/visualization_earthquakes.html



### OK, back to interactive mapping, because that's fun...

It turns out that folium doesn't do much itself.  It is just a wrapper around something called leafletjs.  You can read more about that here:

http://leafletjs.com/index.html

Leaflet is a library in the JavaScript language.  JavaScript is the language used for most web applications.  We could do the same thing using JavaScript and leaflet directly, but then we would have to learn the syntax for another language.  That might not be too hard, but to keep it simple, we'll stick to the python wrapper for now.  It is good to be aware of, though, because if you want more options than folium allows, you can go directly to leaflet.  

What makes this possible is the fact that leaflet has a well-defined API.  That means that we can pass data back and forth, even from a different language.  


### Setup

Start by installing folium using pip.  At a command prompt, type: 

    pip install folium

Hmm...when I tried this on my desktop, I get an error that says: 

    PermissionError: [WinError 5] Access is denied: 'c:\\program files\\anaconda3\\Lib\\site-packages\\folium'
    
It seems that it is trying to install something in the program files directory, which Windows has protected.  This will depend on the security settings on your machine.  If you get this error, open a command prompt as an administrator.  In the windows search bar, type cmd.  When you see the command prompt, right click, and select run as administrator.  

This did the trick, and now I get: 

    Successfully installed folium-0.2.1
    
In addition, let's go to github and clone the folium repository (https://github.com/python-visualization/folium) to our desktop.  This gives us the source code on our local machine.  What we're really interested in is the examples folder, which gives us a bunch of jupyter notebooks showing how to do different stuff.  You are welcome to explore these as needed. 

You also need to install geopandas, which will make it easier to work with goegraphic data.  The pip installer doesn't work (the long explanation is here: http://geoffboeing.com/2014/09/using-geopandas-windows/), so we'll install using anaconda.  Type: 

    conda install -c conda-forge geopandas
 


Getting Started
---------------

To create a base map, simply pass your starting coordinates to Folium:

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[38.034,-84.500])

to display it in your notebook, just ask for the object representation. 

In [ ]:
m

To save it in a file

In [ ]:
m.save('lex.html')

We can use different backgrounds, or tilesets.  Several are built in.  Options include Stamen Terrain, Stamen Toner, Mapbox Bright, and Mapbox Control room tiles. 

In [ ]:
folium.Map(
    location=[38.034,-84.500],
    tiles='Stamen Toner',
    zoom_start=13
)

Pick one you like and work with that for the rest of the class.  

Folium also supports Cloudmade and Mapbox custom tilesets- simply pass your key to the API_key keyword.  These are services where you can buy more backgrounds to make your maps look nice. 

```python
folium.Map(location=[45.5236, -122.6750],
           tiles='Mapbox',
           API_key='your.API.key')
```

### Open flights

Let's go back to our openflight data and make some maps. 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("data/airports.dat", header=None, na_values='\\N')
airports.columns = ["id", "name", "city", "country", "iata", "icao", "latitude", "longitude", "altitude","timezone", "dst", "tz", "type", "source"]

# Read in the airlines data.
airlines = pd.read_csv("data/airlines.dat", header=None, na_values='\\N')
airlines.columns = ["id", "name", "alias", "iata", "icao", "callsign", "country", "active"]

# Read in the routes data.
routes = pd.read_csv("data/routes.dat", header=None, na_values='\\N')
routes.columns = ["airline", "airline_id", "source", "source_id", "dest", "dest_id", "codeshare", "stops", "equipment"]

In [ ]:
# let's peek at what we have
airports.head()

In [ ]:
airlines.head()

In [ ]:
routes.head()

Make a map with the airports on it.

In [ ]:
# since there are a lot of airports, making the map can be slow
# so limit it to US airports
us_airports = airports[airports['country']=='United States']
len(us_airports)

In [ ]:
# Get a basic world map.
# 30 centers the map E-W, and 0 is the equator
airports_map = folium.Map(location=[30, 0], zoom_start=2)

# Loop through the airports, and draw each one as a marker on the map
# popup tells it what to display when you click on it
for name, row in us_airports.iterrows():
    
    # For some reason, this one airport causes issues with the map.
    if row["name"] != "South Pole Station":
        marker = folium.Marker([row["latitude"], row["longitude"]], popup=row['name'])
        marker.add_to(airports_map)
        
# Save it to a file (it's kinda big for the notebook)
airports_map.save('airports.html')

Hmm...it looks like there are airports everywhere!  Let's try again with smaller makers. 

We can also specify the color.  A list of custom colors is available here: 

http://www.w3schools.com/cssref/css_colors.asp

In [ ]:
# over-write the airports_map, rather than just adding more markers to it. 
airports_map = folium.Map(location=[30, 0], zoom_start=2)

# use circle markers this time, with custom size and color
for name, row in us_airports.iterrows():
        
    # For some reason, this one airport causes issues with the map.
    if row["name"] != "South Pole Station":
        marker = folium.CircleMarker([row["latitude"], row["longitude"]], 
                                     radius=5,
                                     color='DarkCyan',
                                     fill_color='DarkCyan', 
                                     popup=row['name'])
        marker.add_to(airports_map)
        
airports_map.save('airports.html')

You can also select icons to use as markers.  That code would look like: 

    marker = folium.Marker([row["latitude"], row["longitude"]], 
                           icon=folium.Icon(icon='cloud'), 
                           popup=row['name'])
                           
The list of icons comes from something called bootstrap, and can be found here: 

http://www.bootstrapicons.com/


Or you can use clusters of markers to clean up the map.  This will group them when you zoom out, similar to a Craigslist map.  You can see how to do that here: 

https://ocefpaf.github.io/python4oceanographers/blog/2015/12/14/geopandas_folium/

You can clean up the rest of this airports map as part of your homework this week.  

Let's draw the routes, but since we have lots, let's just start with the routes departing Lexington. 

In [ ]:
# Select the LEX routes, then join the source airports
lex_routes = routes[(routes['source']=="LEX")]
lex_routes = pd.merge(lex_routes, airports, left_on='source_id', right_on='id', how='left')

In [ ]:
# join the destination airports.  Here we need to use the suffixes option, because 
# the column names overlap, and we want to distinguish between source and dest
lex_routes = pd.merge(lex_routes, airports, 
                      left_on='dest_id', 
                      right_on='id', 
                      how='left', 
                      suffixes=['_source','_dest'])

In [ ]:
# here is what our data looks like
lex_routes

In [ ]:
# It looks like source has some duplicate names.  Drop the values from the airports
# file ane keep the one from the routes file
lex_routes = lex_routes.drop(['source_y','source'], axis=1)
lex_routes = lex_routes.rename(columns={'source_x': 'source'})

In [ ]:
# Let's keep only one route between each airport pair
# so we don't have a bunch of lines on top of each other
# The subset option tells it to consider just those columns when determining
# what is a duplicate. 

lex_routes = lex_routes.drop_duplicates(subset=['source', 'dest'])
lex_routes

That looks better.  Now, let's create a map.  To avoid adding duplicate airports, we are going to use a container called a set.  A set is an unordered collection of unique elements.  This means we can keep adding LEX to the set, and end up with only 1 LEX in the end.  

In [ ]:
# create a basic map, centered on Lexington
lex_air = folium.Map(
    location=[38.034,-84.500],
    tiles='Stamen Toner',
    zoom_start=4
)

In [ ]:
# Define some empty sets
airport_set = set()
route_set = set()

# Make sure we don't add duplicates, especially for the origins
for name, row in lex_routes.iterrows():
    
    if row['source'] not in airport_set: 
        popup_string = row['city_source'] + ' (' + row['source'] + ')'
        marker = folium.CircleMarker([row["latitude_source"], row["longitude_source"]], 
                                     color='DarkCyan',
                                     fill_color='DarkCyan', 
                                     radius=5, popup=popup_string)
        marker.add_to(lex_air)
        airport_set.add(row['source'])
        
    if row['dest'] not in airport_set: 
        popup_string = row['city_dest'] + '(' + row['dest'] + ')'
        marker = folium.CircleMarker([row["latitude_dest"], row["longitude_dest"]], 
                                     color='MidnightBlue',
                                     fill_color='MidnightBlue', 
                                     radius=5, popup=popup_string)
        marker.add_to(lex_air)
        airport_set.add(row['dest'])
    
    # the parentheses in the indicate that we are adding a tuple to the route_set
    if (row['source'],row['dest']) not in route_set:            
        popup_string = row['source'] + '-' + row['dest']        
        line = folium.PolyLine([(row["latitude_source"], row["longitude_source"]), 
                                (row["latitude_dest"], row["longitude_dest"])], 
                                weight=2, 
                                popup=popup_string)
        line.add_to(lex_air)
        route_set.add((row['source'],row['dest']))
        
lex_air

That's cool.  But airplanes don't fly in a straight line.  They follow the great circle.  So when you fly from Chicago to London, you go over Greenland (which is really pretty on a clear day!).  Can we make the lines follow a great circle? 

It looks like there are some options here: 

http://gis.stackexchange.com/questions/47/what-tools-in-python-are-available-for-doing-great-circle-distance-line-creati

Let's try one of them. 

In [ ]:
import pyproj

# when creating a function, it is good practice to define the API!
def getGreatCirclePoints(startlat, startlon, endlat, endlon): 
    """
    startlat - starting latitude 
    startlon - starting longitude 
    endlat   - ending latitude 
    endlon   - ending longitude 
    
    returns - a list of tuples, where each tuple is the lat-long for a point
              along the curve.  
    """
    # calculate distance between points
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(startlon, startlat, endlon, endlat)

    # calculate line string along path with segments <= 20 km
    lonlats = g.npts(startlon, startlat, endlon, endlat,
                     1 + int(dist / 20000))

    # the npts function uses lon-lat, while the folium functions use lat-lon
    # This sort of thing is maddening!  What happens is the lines don't show
    # up on the map and you don't know why.  Learn from my mistakes
    latlons = []
    for lon_lat in lonlats: 
        
        # this is how you get values out of a tuple
        (lon, lat) = lon_lat
        
        # add them to our list
        latlons.append((lat, lon)) 
    
    # npts doesn't include start/end points, so prepend/append them
    latlons.insert(0, (startlat, startlon))
    latlons.append((endlat, endlon))
    
    return latlons


In [ ]:
# any time we write a function, we should test that it works
p = getGreatCirclePoints(38.034, -84.500, 33.636700, -84.428101) 
p

In [ ]:
# create a basic map, centered on Lexington
lex_air = folium.Map(
    location=[38.034,-84.500],
    tiles='Stamen Toner',
    zoom_start=4
)

In [ ]:
# define the map in the same way, but use great circles for the lines

# Define some empty sets
airport_set = set()
route_set = set()

# Make sure we don't add duplicates, especially for the origins
for name, row in lex_routes.iterrows():
    
    if row['source'] not in airport_set: 
        popup_string = row['city_source'] + ' (' + row['source'] + ')'
        marker = folium.CircleMarker([row["latitude_source"], row["longitude_source"]], 
                                     color='DarkCyan',
                                     fill_color='DarkCyan', 
                                     radius=5, popup=popup_string)
        marker.add_to(lex_air)
        airport_set.add(row['source'])
        
    if row['dest'] not in airport_set: 
        popup_string = row['city_dest'] + '(' + row['dest'] + ')'
        marker = folium.CircleMarker([row["latitude_dest"], row["longitude_dest"]], 
                                     color='MidnightBlue',
                                     fill_color='MidnightBlue', 
                                     radius=5, popup=popup_string)
        marker.add_to(lex_air)
        airport_set.add(row['dest'])
    
    # PolyLine will accept a whole list of tuples, not just two
    if (row['source'],row['dest']) not in route_set:            
        popup_string = row['source'] + '-' + row['dest']       
        
        gc_points = getGreatCirclePoints(row["latitude_source"], 
                                         row["longitude_source"], 
                                         row["latitude_dest"], 
                                         row["longitude_dest"])
        
        line = folium.PolyLine(gc_points, weight=2, popup=popup_string)
        line.add_to(lex_air)
        route_set.add((row['source'],row['dest']))
        
lex_air   

In [ ]:
# save it to its own file
lex_air.save("lex_air.html")

### Your turn

The above map shows everywhere you can get to from Lexington on a direct flight.  Your job is to:

1. Make a map of all the possible destinations with one transfer. 
2. Make a map of all the possible desitnations with two transfers. 

Make the maps look nice!  Use color coding, vary the size of the features, or be selective about what you display in order to communicate the information effectively.  

Bonus: This is the air travel version of the Kevin Bacon game (https://oracleofbacon.org/).  What is the number N, such that you can reach every airport in the world with N or fewer transfers?  

Extra Bonus: Use this very important piece of knowledge to impress your friends at parties!